In [2]:
#Use ggplot for styling.

#importing data
import csv
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import DB


2017-04-08 00:55:36,533 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-08 00:55:36,535 INFO sqlalchemy.engine.base.Engine ()
2017-04-08 00:55:36,537 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-08 00:55:36,538 INFO sqlalchemy.engine.base.Engine ()
2017-04-08 00:55:36,539 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("access_log")
2017-04-08 00:55:36,540 INFO sqlalchemy.engine.base.Engine ()
2017-04-08 00:55:36,542 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("account_company")
2017-04-08 00:55:36,543 INFO sqlalchemy.engine.base.Engine ()
2017-04-08 00:55:36,544 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("account_external")
2017-04-08 00:55:36,544 INFO sqlalchemy.engine.base.Engine ()
2017-04-08 00:55:36,546 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("authorization_level")
2017-04-08 00:55:36,547 INFO sqlalchemy.engine.base.Engine ()
2017-0

In [5]:
######################################
#Get # vendors with incomplete details.
######################################


#Vendors did not yield any empty entries
vendors = []
vendor_addr = []
addr_header = None

#SQL-esque way:
#JOIN vendors and vendor address on the keys (vendor_id).
#Search rows for empty fields and increment.
#Iterate through vendors 


#Archaic approach as it took some time to read in.
with open('data/vendor.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    next(reader) #skip header
    vendors = [row for row in reader]

    
with open('data/vendor_address.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    addr_header = next(reader) #skip header
    vendor_addr = [row for row in reader]

counter = 0
vendor_ids = set()
inc_vendors = []

for idx,v in enumerate(vendors):
    for entry in v:
        #print(entry)
        if not entry:
            counter += 1
            vendor_ids.add(v[0])
            inc_vendors.append(v)
            break
            
print(counter)

for idx,v in enumerate(vendor_addr):
    for entry in v:
        if not entry and v[0] not in vendor_ids:
            counter += 1
            v_2 = []
            for e in v:
                if not e:
                    v_2.append("EMPTY")
                else:
                    v_2.append(e)
                    
            inc_vendors.append(v_2)
            vendor_ids.add(v[0])
            break
            
print(counter)
print(vendor_ids)
print(inc_vendors)

with open('analysis/incomplete_vendors.csv','w') as out_file:
    out_file.write("\n"+",".join(addr_header))
    for v in inc_vendors:
        out_file.write("\n"+",".join(v))

0
10
{'VDA-00547', 'VDA-00449', 'VDA-00597', 'VDA-00773', 'VDA-00772', 'VDA-00346', 'VDA-00398', 'VDA-00109', 'VDA-00323', 'VDA-00206'}
[['VDA-00547', 'VD-0483', 'Lewis, Shelly', '1', '2011-07-19 00:00:00', 'Stauffacherstrasse 60', 'EMPTY', '8004', 'Switzerland'], ['VDA-00323', 'VD-0211', 'Guthrie, Jonah', '1', '2005-05-19 00:00:00', 'Kindhausenstrasse 7', 'EMPTY', '8602', 'Switzerland'], ['VDA-00398', 'VD-0305', 'Mcclain, Elmo', '1', '2012-07-18 00:00:00', 'R??ffelstrasse 32', 'EMPTY', '8045', 'Switzerland'], ['VDA-00346', 'VD-0237', 'Webb, Sierra', '1', '2015-07-10 00:00:00', 'Lindauerstrasse 27', 'Tagelswangen', 'EMPTY', 'EMPTY'], ['VDA-00597', 'VD-0546', 'Bradford, Ulric', '1', '2014-09-01 00:00:00', 'Zugerstrasse 23', 'W??denswil', 'EMPTY', 'EMPTY'], ['VDA-00206', 'VD-0065', 'Morse, Carolyn', '1', '2015-04-09 00:00:00', 'bas du Village 13', 'Les Bois', 'EMPTY', 'EMPTY'], ['VDA-00773', 'VD-0571', 'Mcfadden, Stewart', '2', '2011-11-02 00:00:00', 'Im Baumgarten 35', 'EMPTY', '8332', 

In [18]:
########################
#Counting split payments
########################
from sqlalchemy import func

#Perform query to get repeating narratives (grouped count > 1)
S = DB.sess
query = S.query(func.count(DB.Transactions.narrative), DB.Transactions.narrative).\
group_by(DB.Transactions.narrative).\
having(func.count(DB.Transactions.narrative) > 1)

results = []

#Query for each repeated narrative
for row in query:
    invoice_query = S.query(DB.Transactions).filter_by(narrative=row.narrative)
    for r in invoice_query:
        results.append(str(r))

print(results)

with open('analysis/split_payments.csv','w') as out:
    #out.write(",".join(trans_header)+"\n")
    for t in results:
        out.write(t+"\n")

2017-04-08 01:54:25,981 INFO sqlalchemy.engine.base.Engine SELECT count(transactions.narrative) AS count_1, transactions.narrative AS transactions_narrative 
FROM transactions GROUP BY transactions.narrative 
HAVING count(transactions.narrative) > ?
2017-04-08 01:54:25,983 INFO sqlalchemy.engine.base.Engine (1,)
2017-04-08 01:54:26,009 INFO sqlalchemy.engine.base.Engine SELECT transactions.transaction_id AS transactions_transaction_id, transactions.transaction_ref AS transactions_transaction_ref, transactions.transaction_dt AS transactions_transaction_dt, transactions.amount AS transactions_amount, transactions.debit_acct AS transactions_debit_acct, transactions.credit_acct AS transactions_credit_acct, transactions.status AS transactions_status, transactions.narrative AS transactions_narrative, transactions.approval_dt AS transactions_approval_dt, transactions.approved_by_user_id AS transactions_approved_by_user_id 
FROM transactions 
WHERE transactions.narrative = ?
2017-04-08 01:54:2

In [4]:
##################################################################################
#Get consecutive Purchase Order IDs, considered only if they have the same vendor
#################################################################################

#Select 2 columns: po_id and vendor_id
S = DB.sess
query = S.query(DB.InvoicePurchaseOrder)
purchase_IDs = {}
purch = []

#Parse po_id and pair with vendor_id
for row in query:
    raw_ID = int(row.po_id.split("-")[1])
    purch.append(raw_ID)
    purchase_IDs[raw_ID] = row.vendor_id

purch = sorted(purch)

consecutives = 0
consec = set()

#Reduce to consecutive po_ids
for po_ID in purchase_IDs:
    next_ID = po_ID + 1
    
    if next_ID in purchase_IDs:
        consec.add(po_ID)
        consec.add(next_ID)

cons = sorted(consec)
counter = 0
counted = set()

print(cons)

#Find grouped vendors.
for idx,element in enumerate(cons[:len(cons)-1]):
    if cons[idx] not in counted:
        if cons[idx]+1 == cons[idx+1] and purchase_IDs[cons[idx]] == purchase_IDs[cons[idx+1]]:
            print(cons[idx],cons[idx+1],purchase_IDs[cons[idx]],purchase_IDs[cons[idx+1]])
            counter +=1
            counted.add(cons[idx])
            if cons[idx+1] not in counted:
                counter+=1
                counted.add(cons[idx+1])

print("Counted Consecutives with common Vendor:",counter)

#Is 73. Should be 75. Check edge cases.
#Now I get 80 after fixing it. Let's just agree that it makes no sense.


results = []
for element in counted:
    query = S.query(DB.InvoicePurchaseOrder).filter_by(po_id="PUO-"+str(element).zfill(9))
    shady_query = S.query(DB.InvoicePurchaseOrder).filter_by(po_id="PUO-"+str(element))
    for r in query:
        print(r)
        results.append(str(r))
    for r in shady_query:
        results.append(str(r))

with open('analysis/consecutive_purchase_orders.csv','w') as out:
    #out.write(",".join(trans_header)+"\n")
    for t in results:
        out.write(t+"\n")


2017-04-08 00:55:57,545 INFO sqlalchemy.engine.base.Engine SELECT invoice_purchase_order.invoice_id AS invoice_purchase_order_invoice_id, invoice_purchase_order.po_id AS invoice_purchase_order_po_id, invoice_purchase_order.vendor_id AS invoice_purchase_order_vendor_id, invoice_purchase_order.invoice_dt AS invoice_purchase_order_invoice_dt, invoice_purchase_order.invoice_image_path AS invoice_purchase_order_invoice_image_path, invoice_purchase_order.payment_due_by_dt AS invoice_purchase_order_payment_due_by_dt, invoice_purchase_order.received_dt AS invoice_purchase_order_received_dt, invoice_purchase_order.received_by_user_id AS invoice_purchase_order_received_by_user_id, invoice_purchase_order.total AS invoice_purchase_order_total 
FROM invoice_purchase_order
2017-04-08 00:55:57,546 INFO sqlalchemy.engine.base.Engine ()
[709771, 709772, 709773, 709774, 709775, 709776, 709777, 709778, 709779, 709780, 709781, 709782, 709783, 709784, 709785, 709786, 709787, 709788, 709789, 709790, 709791,

In [11]:
################################################
#Actual amount frauded (includes 100 at start?)
################################################
from sqlalchemy.sql import func
S = DB.sess
query = S.query(func.sum(DB.InvoicePurchaseOrder.total)).filter_by(received_by_user_id="u8284") #account used for fraud.

for row in query:
    print("Total amount used for fraud:",row)
#The initial 100 were never actually transferred out of the company.
#The remaining 1015000, on the other hand, were.

2017-04-08 01:04:26,943 INFO sqlalchemy.engine.base.Engine SELECT sum(invoice_purchase_order.total) AS sum_1 
FROM invoice_purchase_order 
WHERE invoice_purchase_order.received_by_user_id = ?
2017-04-08 01:04:26,944 INFO sqlalchemy.engine.base.Engine ('u8284',)
Total amount used for fraud: (1015100.0,)


In [17]:
#######################################
#Amount of suspicious invoice approvals
#######################################
from sqlalchemy import func

#Query for all transactions approved by shady user.
#Group by invoice due to split payments
#Count resulting amount (should be 26, but is 25 for whatever reason.)
S = DB.sess
query = S.query(func.count(DB.Transactions.narrative)).filter_by(approved_by_user_id="u8284").group_by(DB.Transactions.narrative)

counter = 0
for row in query:
    counter +=1

print("Counted invoices:",counter)

2017-04-08 01:11:34,772 INFO sqlalchemy.engine.base.Engine SELECT count(transactions.narrative) AS count_1 
FROM transactions 
WHERE transactions.approved_by_user_id = ? GROUP BY transactions.narrative
2017-04-08 01:11:34,773 INFO sqlalchemy.engine.base.Engine ('u8284',)
Counted invoices: 25
